# GVC participation rate

This notebook computes the trade-based and production-based GVC participation rates at the country-level and the 5-, 15-, and 35-sector levels. It uses the trade accounting framework of Borin and Mancini (2019). Results are saved in `data/`.

The trade-based GVC participation rate is defined as
```
GVCP_trade_f = (REX1 + REX2 + REX3 + REF1 + REF2) / Exports
GVCP_trade_f = (FVA + PDC1 + PDC2) / Exports
GVCP_trade = GVCP_trade_f + GVCP_trade_b
```
where sectors are broken down by export sectors.

The production-based GVC participation rate, meanwhile, is defined as
```
GVCP_prod = (DAVAX2 + REX1 + REX2 + REX3 + REF1 + REF2) / va
```
where sectors are broken down by origin sectors.

## Set up

In [1]:
import pandas as pd
import duckdb

In [2]:
ta, summary, output = 'ta.parquet', 'summary.parquet', 'gvcp.parquet'
# ta, summary, output = 'ta62.parquet', 'summary62.parquet', 'gvcp62.parquet'
# ta, summary, output = 'ta62-const.parquet', 'summary62-const.parquet', 'gvcp62-const.parquet'

## Load and process data

### Breakdown by export sectors

In [3]:
duckdb.sql(
    f"""
    (SELECT t, s, 0 AS agg, 0 AS i, 
        sum(exports) AS exports, 
        sum(davax1) AS davax1,
        sum(davax2) AS davax2,
        sum(rex1) AS rex1,
        sum(rex2) AS rex2,
        sum(rex3) AS rex3,
        sum(ref1) AS ref1,
        sum(ref2) AS ref2,
        sum(fva) AS fva,
        sum(pdc1) AS pdc1,
        sum(pdc2) AS pdc2
    FROM '../data/{ta}' WHERE breakdown='es'
    GROUP BY t, s
    ORDER BY t, s)
    """
).df()

,t,s,agg,i,exports,davax1,davax2,rex1,rex2,rex3,ref1,ref2,fva,pdc1,pdc2
0,2017,1,0,0,3.070807e+05,38064.715576,146914.865860,23452.827391,51495.749711,2052.356854,884.930743,1575.689237,41963.673237,583.185459,92.687898
1,2017,2,0,0,2.012666e+05,47110.961255,41083.501794,17074.845063,24110.831991,1003.120197,312.696984,370.736909,69486.698617,449.386055,263.851400
2,2017,3,0,0,4.244186e+05,68172.496630,67085.192026,22125.903230,40049.045953,1494.150823,603.227698,737.891131,220026.841389,1737.674891,2386.208790
3,2017,4,0,0,3.922382e+04,7637.244193,9032.701360,2680.435067,4477.817455,169.797777,12.487047,13.152135,15169.491296,16.349996,14.344661
4,2017,5,0,0,2.582087e+05,67620.138171,103573.523589,21387.495786,28173.996205,1220.126075,438.850383,597.135926,35036.582117,137.413035,23.436808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433,2022,69,0,0,1.132128e+05,28018.509963,42395.608822,5288.216464,27411.538563,1539.355478,16.088688,45.726810,8478.262526,18.386472,1.136633
434,2022,70,0,0,4.088153e+05,55108.101371,195246.873850,28744.513829,103151.855110,5139.292740,406.677978,925.679293,19960.770585,125.524797,6.007461
435,2022,71,0,0,3.200703e+05,91443.015455,113722.158254,19324.495160,45587.703595,1784.911693,738.552394,260.179120,46800.810049,373.964750,34.461052
436,2022,72,0,0,5.751075e+04,26119.672015,15264.379160,2465.520426,4001.553369,136.046369,22.570661,21.110329,9470.543161,7.810609,1.546191


In [ ]:
df_es = duckdb.sql(
    f"""
    (SELECT t, s, 0 AS agg, 0 AS i, 
        sum(exports) AS exports, 
        sum(davax1) AS davax1,
        sum(davax2) AS davax2,
        sum(rex1) AS rex1,
        sum(rex2) AS rex2,
        sum(rex3) AS rex3,
        sum(ref1) AS ref1,
        sum(ref2) AS ref2,
        sum(fva) AS fva,
        sum(pdc1) AS pdc1,
        sum(pdc2) AS pdc2
    FROM '../data/{ta}' WHERE breakdown='es'
    GROUP BY t, s
    ORDER BY t, s)

    UNION ALL

    (SELECT t, s, 5 AS agg, i5 AS i, 
        sum(exports) AS exports, 
        sum(davax1) AS davax1,
        sum(davax2) AS davax2,
        sum(rex1) AS rex1,
        sum(rex2) AS rex2,
        sum(rex3) AS rex3,
        sum(ref1) AS ref1,
        sum(ref2) AS ref2,
        sum(fva) AS fva,
        sum(pdc1) AS pdc1,
        sum(pdc2) AS pdc2
    FROM read_parquet('../data/{ta}') WHERE breakdown='es'
    GROUP BY t, s, i5
    ORDER BY t, s, i5)

    UNION ALL
    
    (SELECT t, s, 15 AS agg, i15 AS i, 
        sum(exports) AS exports, 
        sum(davax1) AS davax1,
        sum(davax2) AS davax2,
        sum(rex1) AS rex1,
        sum(rex2) AS rex2,
        sum(rex3) AS rex3,
        sum(ref1) AS ref1,
        sum(ref2) AS ref2,
        sum(fva) AS fva,
        sum(pdc1) AS pdc1,
        sum(pdc2) AS pdc2
    FROM read_parquet('../data/{ta}') WHERE breakdown='es'
    GROUP BY t, s, i15
    ORDER BY t, s, i15)
    
    UNION ALL
    
    (SELECT t, s, 35 AS agg, i, 
        sum(exports) AS exports, 
        sum(davax1) AS davax1,
        sum(davax2) AS davax2,
        sum(rex1) AS rex1,
        sum(rex2) AS rex2,
        sum(rex3) AS rex3,
        sum(ref1) AS ref1,
        sum(ref2) AS ref2,
        sum(fva) AS fva,
        sum(pdc1) AS pdc1,
        sum(pdc2) AS pdc2
    FROM read_parquet('../data/{ta}') WHERE breakdown='es'
    GROUP BY t, s, i
    ORDER BY t, s, i)
    """
).df()

df_es['gvc_trade_f'] = df_es['rex1'] + df_es['rex2'] + df_es['rex3'] + df_es['ref1'] + df_es['ref2']
df_es['gvc_trade_b'] = df_es['fva'] + df_es['pdc1'] + df_es['pdc2']
df_es['gvc_trade'] = df_es['gvc_trade_f'] + df_es['gvc_trade_b']
df_es['t'] = df_es['t'].astype(int)
df_es = df_es[['t', 's', 'agg', 'i', 'exports', 'gvc_trade_f', 'gvc_trade_b', 'gvc_trade']]

### Breakdown by origin sectors

In [ ]:
df_os = duckdb.sql(
    f"""
    (SELECT t, s, 0 AS agg, 0 AS i, 
        sum(davax1) AS davax1,
        sum(davax2) AS davax2,
        sum(rex1) AS rex1,
        sum(rex2) AS rex2,
        sum(rex3) AS rex3,
        sum(ref1) AS ref1,
        sum(ref2) AS ref2,
        sum(fva) AS fva,
        sum(pdc1) AS pdc1,
        sum(pdc2) AS pdc2
    FROM read_parquet('../data/{ta}') WHERE breakdown='os'
    GROUP BY t, s
    ORDER BY t, s)

    UNION ALL

    (SELECT t, s, 5 AS agg, i5 AS i, 
        sum(davax1) AS davax1,
        sum(davax2) AS davax2,
        sum(rex1) AS rex1,
        sum(rex2) AS rex2,
        sum(rex3) AS rex3,
        sum(ref1) AS ref1,
        sum(ref2) AS ref2,
        sum(fva) AS fva,
        sum(pdc1) AS pdc1,
        sum(pdc2) AS pdc2
    FROM read_parquet('../data/{ta}') WHERE breakdown='os'
    GROUP BY t, s, i5
    ORDER BY t, s, i5)

    UNION ALL
    
    (SELECT t, s, 15 AS agg, i15 AS i, 
        sum(davax1) AS davax1,
        sum(davax2) AS davax2,
        sum(rex1) AS rex1,
        sum(rex2) AS rex2,
        sum(rex3) AS rex3,
        sum(ref1) AS ref1,
        sum(ref2) AS ref2,
        sum(fva) AS fva,
        sum(pdc1) AS pdc1,
        sum(pdc2) AS pdc2
    FROM read_parquet('../data/{ta}') WHERE breakdown='os'
    GROUP BY t, s, i15
    ORDER BY t, s, i15)
    
    UNION ALL
    
    (SELECT t, s, 35 AS agg, i, 
        sum(davax1) AS davax1,
        sum(davax2) AS davax2,
        sum(rex1) AS rex1,
        sum(rex2) AS rex2,
        sum(rex3) AS rex3,
        sum(ref1) AS ref1,
        sum(ref2) AS ref2,
        sum(fva) AS fva,
        sum(pdc1) AS pdc1,
        sum(pdc2) AS pdc2
    FROM read_parquet('../data/{ta}') WHERE breakdown='os'
    GROUP BY t, s, i
    ORDER BY t, s, i)
    """
).df()

df_os['gvc_prod'] = df_os['davax2'] + df_os['rex1'] + df_os['rex2'] + df_os['rex3'] + df_os['ref1'] + df_os['ref2']
df_os['t'] = df_os['t'].astype(int)
df_os = df_os[['t', 's', 'agg', 'i', 'gvc_prod']]

### Value added

In [ ]:
va = duckdb.sql(
    f"""
    (SELECT t, s, 0 AS agg, 0 AS i, sum(va) AS va, 
    FROM read_parquet('../data/{summary}')
    GROUP BY t, s
    ORDER BY t, s)

    UNION ALL

    (SELECT t, s, 5 AS agg, i5 AS i, sum(va) AS va, 
    FROM read_parquet('../data/{summary}')
    GROUP BY t, s, i5
    ORDER BY t, s, i5)

    UNION ALL

    (SELECT t, s, 15 AS agg, i15 AS i, sum(va) AS va, 
    FROM read_parquet('../data/{summary}')
    GROUP BY t, s, i15
    ORDER BY t, s, i15)

    UNION ALL

    (SELECT t, s, 35 AS agg, i, sum(va) AS va, 
    FROM read_parquet('../data/{summary}')
    GROUP BY t, s, i
    ORDER BY t, s, i)
    """
).df()

In [ ]:
df_os = pd.merge(df_os, va)

## Consolidate and save

In [ ]:
df = pd.merge(df_es, df_os)
df['gvcp_trade_f'] = df['gvc_trade_f'] / df['exports']
df['gvcp_trade_b'] = df['gvc_trade_b'] / df['exports']
df['gvcp_trade'] = df['gvc_trade'] / df['exports']
df['gvcp_prod'] = df['gvc_prod'] / df['va']

df = df[[
    't', 's', 'agg', 'i', 'exports', 'va',
    'gvc_trade_f', 'gvc_trade_b', 'gvc_trade', 'gvc_prod',
    'gvcp_trade_f', 'gvcp_trade_b', 'gvcp_trade', 'gvcp_prod'
]]

df.to_parquet(f'../data/{output}', index=False)

### View results

In [ ]:
duckdb.sql(f"SELECT * FROM read_parquet('../data/{output}')").df()